# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from pandas import read_sql_table
import pickle


import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nicholasreimold/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nicholasreimold/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nicholasreimold/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df=read_sql_table('DisasterResponse', engine)
X=df['message']
y=df.drop(['message','id','original','genre'],axis=1)

y.head()


ValueError: Table DisasterResponse not found

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    """
    Tokenize and parse text field
    
    input: text that needs to be parsed
    Output: tokens extracted from text
    """
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('RF', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
def perf_report(model, X_test, y_test):
    '''
    Create classificartion_report from trained model
    Input: Trained Model and test set
    Output: sklearn's classification report results
    '''
    y_pred = model.predict(X_test)
    
    for jj, column in enumerate(y_test):
        print(column)
        print(classification_report(y_test[column], y_pred[:, jj]))

In [10]:
perf_report(pipeline, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.70      0.46      0.55      1243
           1       0.85      0.93      0.89      3958
           2       0.32      0.49      0.39        43

    accuracy                           0.82      5244
   macro avg       0.62      0.63      0.61      5244
weighted avg       0.81      0.82      0.80      5244

request
              precision    recall  f1-score   support

           0       0.90      0.97      0.94      4345
           1       0.80      0.49      0.61       899

    accuracy                           0.89      5244
   macro avg       0.85      0.73      0.77      5244
weighted avg       0.88      0.89      0.88      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        25

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      524

/Users/nicholasreimold/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5178
           1       0.00      0.00      0.00        66

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.97      0.99      0.98      5244

other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5019
           1       0.33      0.00      0.01       225

    accuracy                           0.96      5244
   macro avg       0.65      0.50      0.49      5244
weighted avg       0.93      0.96      0.94      5244

weather_related
              precision    recall  f1-score   support

           0       0.90      0.96      0.93      3807
           1       0.86      0.71      0.78      1437

    accuracy                           0.89      5244
   macro avg       0.88      0.83      0.85      5244
weighted avg       0.89      0.89    

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
param_grid =  {'tfidf__use_idf': (True, False), 
              'RF__estimator__n_estimators': [5, 10,20,40,80,100],
              'RF__estimator__max_depth' : [2,4,6,8]} 

cv = GridSearchCV(pipeline, param_grid=param_grid)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
cv.fit(X_train, y_train)
perf_report(cv, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1243
           1       0.75      1.00      0.86      3958
           2       0.00      0.00      0.00        43

    accuracy                           0.75      5244
   macro avg       0.25      0.33      0.29      5244
weighted avg       0.57      0.75      0.65      5244

request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      4345
           1       0.00      0.00      0.00       899

    accuracy                           0.83      5244
   macro avg       0.41      0.50      0.45      5244
weighted avg       0.69      0.83      0.75      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        25

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      524

/Users/nicholasreimold/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5178
           1       0.00      0.00      0.00        66

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.97      0.99      0.98      5244

other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5019
           1       0.00      0.00      0.00       225

    accuracy                           0.96      5244
   macro avg       0.48      0.50      0.49      5244
weighted avg       0.92      0.96      0.94      5244

weather_related
              precision    recall  f1-score   support

           0       0.73      1.00      0.84      3807
           1       1.00      0.00      0.00      1437

    accuracy                           0.73      5244
   macro avg       0.86      0.50      0.42      5244
weighted avg       0.80      0.73    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
pipeline_improved = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('KNN', MultiOutputClassifier(KNeighborsClassifier()))
])

In [14]:
#Train & predict
pipeline_improved.fit(X_train, y_train)
perf_report(pipeline_improved, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.71      0.10      0.18      1243
           1       0.77      0.05      0.09      3958
           2       0.01      0.93      0.02        43

    accuracy                           0.07      5244
   macro avg       0.50      0.36      0.09      5244
weighted avg       0.75      0.07      0.11      5244

request
              precision    recall  f1-score   support

           0       0.84      1.00      0.91      4345
           1       0.81      0.05      0.10       899

    accuracy                           0.84      5244
   macro avg       0.82      0.52      0.50      5244
weighted avg       0.83      0.84      0.77      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        25

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      524

/Users/nicholasreimold/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.73      1.00      0.84      3807
           1       0.78      0.04      0.08      1437

    accuracy                           0.73      5244
   macro avg       0.76      0.52      0.46      5244
weighted avg       0.75      0.73      0.64      5244

floods
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      4811
           1       1.00      0.01      0.02       433

    accuracy                           0.92      5244
   macro avg       0.96      0.50      0.49      5244
weighted avg       0.92      0.92      0.88      5244

storm
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      4765
           1       0.60      0.01      0.01       479

    accuracy                           0.91      5244
   macro avg       0.75      0.50      0.48      5244
weighted avg       0.88      0.91      0.87      5244

fire
 

In [15]:
param_grid =  {'tfidf__use_idf': (True, False), 
              'KNN__estimator__n_neighbors': [2,5,8,11]} 

cv_improved = GridSearchCV(pipeline_improved, param_grid=param_grid)

In [16]:
cv_improved.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [17]:
perf_report(cv_improved, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.63      0.44      0.52      1243
           1       0.84      0.91      0.87      3958
           2       0.33      0.42      0.37        43

    accuracy                           0.80      5244
   macro avg       0.60      0.59      0.59      5244
weighted avg       0.78      0.80      0.78      5244

request
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      4345
           1       0.86      0.23      0.37       899

    accuracy                           0.86      5244
   macro avg       0.86      0.61      0.65      5244
weighted avg       0.86      0.86      0.83      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        25

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      524

/Users/nicholasreimold/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5019
           1       0.50      0.00      0.01       225

    accuracy                           0.96      5244
   macro avg       0.73      0.50      0.49      5244
weighted avg       0.94      0.96      0.94      5244

weather_related
              precision    recall  f1-score   support

           0       0.76      0.98      0.86      3807
           1       0.80      0.18      0.29      1437

    accuracy                           0.76      5244
   macro avg       0.78      0.58      0.57      5244
weighted avg       0.77      0.76      0.70      5244

floods
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      4811
           1       0.53      0.05      0.10       433

    accuracy                           0.92      5244
   macro avg       0.73      0.52      0.53      5244
weighted avg       0.89      0.92      0.89      52

### 9. Export your model as a pickle file

In [20]:
##knn looks to do slightly better, so we choose that file
with open('model.pkl', 'wb') as f:
    pickle.dump(cv_improved, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.